## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-02-16-48-50 +0000,nyt,"Cue the Sun: This Lab Recreates Hot, Sweaty Da...",https://www.nytimes.com/2025/09/02/climate/res...
1,2025-09-02-16-48-42 +0000,nypost,Lefty MSNBC analyst who complained about DC cr...,https://nypost.com/2025/09/02/us-news/msnbc-an...
2,2025-09-02-16-48-26 +0000,bbc,North Korea's Kim arrives in Beijing with daug...,https://www.bbc.com/news/articles/c78z2p6gg1zo...
3,2025-09-02-16-47-35 +0000,bbc,England given brutal reality check by South Af...,https://www.bbc.com/sport/cricket/articles/ceq...
4,2025-09-02-16-47-03 +0000,wapo,Brazilian justice opens Bolsonaro’s trial with...,https://www.washingtonpost.com/world/2025/09/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,46
218,china,16
282,new,13
358,earthquake,10
670,xi,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
249,2025-09-01-20-19-52 +0000,nypost,India’s leader Modi cozies up to Putin and Xi ...,https://nypost.com/2025/09/01/world-news/india...,110
243,2025-09-01-20-43-57 +0000,nypost,Trump awarding Presidential Medal of Freedom t...,https://nypost.com/2025/09/01/us-news/trump-aw...,95
210,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...,95
10,2025-09-02-16-36-26 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...,92
67,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,91


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
249,110,2025-09-01-20-19-52 +0000,nypost,India’s leader Modi cozies up to Putin and Xi ...,https://nypost.com/2025/09/01/world-news/india...
243,49,2025-09-01-20-43-57 +0000,nypost,Trump awarding Presidential Medal of Freedom t...,https://nypost.com/2025/09/01/us-news/trump-aw...
210,49,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...
269,39,2025-09-01-18-23-50 +0000,nypost,Copper hits one-month peak on strong China fac...,https://nypost.com/2025/09/01/business/copper-...
75,39,2025-09-02-14-00-58 +0000,nypost,Trump’s use of National Guard during LA immigr...,https://nypost.com/2025/09/02/us-news/trumps-u...
248,36,2025-09-01-20-20-50 +0000,nypost,"TikTok influencer and her family found dead, w...",https://nypost.com/2025/09/01/world-news/influ...
60,35,2025-09-02-14-37-16 +0000,nypost,Venezuela President Nicolás Maduro claims US s...,https://nypost.com/2025/09/02/world-news/madur...
23,33,2025-09-02-16-10-43 +0000,nypost,Texas homeowner charged after 11-year-old boy ...,https://nypost.com/2025/09/02/us-news/homeowne...
222,33,2025-09-01-22-29-23 +0000,bbc,Watch: Buildings destroyed and rescue efforts ...,https://www.bbc.com/news/videos/cqxgx8d14wwo?a...
242,32,2025-09-01-20-46-30 +0000,nypost,IDF chief pushes Netanyahu to vote on hostage ...,https://nypost.com/2025/09/01/world-news/idf-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
